In [11]:
#Twitter tennis
#Dynamic Graph Temporal Signals
import torch
from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader=TwitterTennisDatasetLoader()
dataset=loader.get_dataset()

In [12]:
#Split
train,test=temporal_signal_split(dataset,train_ratio=0.5)

In [13]:
#Model
#Gatted Recurrent Unit
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch.nn import Module,Linear

class DynamicTemporalNetwork(Module):
    def __init__(self,features,filters):
        super(DynamicTemporalNetwork,self).__init__()

        self.conv_gru=GConvGRU(in_channels=features,out_channels=filters,K=3,normalization='sym')
        self.linear=Linear(in_features=filters,out_features=1)
    
    def forward(self,x,edge_index,edge_weight):
        x=self.conv_gru(x,edge_index,edge_weight).relu()
        x=self.linear(x)

        return x


In [14]:
from torch.nn import MSELoss
from tqdm import tqdm

model=DynamicTemporalNetwork(features=16,filters=32)
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001,betas=(0.9,0.999))
loss=MSELoss()

In [15]:
#Training loop
def training_loop():
    model.train(True)

    NUM_EPOCHS=50

    for epoch in tqdm(range(NUM_EPOCHS)):
        train_loss=0
        test_loss=0
        #Train
        for t,snap in enumerate(train):
            op=model(snap.x,snap.edge_index,snap.edge_weight)
            snap_loss=loss(snap.y,op)

            #training
            model.zero_grad()

            snap_loss.backward()
            optimizer.step()

            #aggregate
            train_loss+=snap_loss.item()

        #Validation
        model.eval()
        
        with torch.no_grad():
            for t,snap in enumerate(test):
                op=model(snap.x,snap.edge_index,snap.edge_weight)
                snap_loss=loss(snap.y,op)

                test_loss+=snap_loss.item()
        
            print("Epoch: {ep}".format(ep=epoch+1))
            print("Train Loss: {l}".format(l=train_loss/(t+1)))
            print("Validation Loss: {l}".format(l=test_loss/(t+1)))


In [16]:
training_loop()

  0%|          | 0/50 [00:00<?, ?it/s]

/Users/smudge/Desktop/Code/temporal-graph-seminar/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  2%|▏         | 1/50 [00:00<00:25,  1.91it/s]

Epoch: 1
Train Loss: 0.4590473581105471
Validation Loss: 0.42696451768279076


  4%|▍         | 2/50 [00:01<00:24,  1.99it/s]

Epoch: 2
Train Loss: 0.4497845354179541
Validation Loss: 0.42692563260595


...

Epoch: 48
Train Loss: 0.44967175846298535
Validation Loss: 0.4269358863433202


 98%|█████████▊| 49/50 [00:27<00:00,  1.72it/s]

Epoch: 49
Train Loss: 0.4496698878705502
Validation Loss: 0.426937185972929


100%|██████████| 50/50 [00:27<00:00,  1.79it/s]

Epoch: 50
Train Loss: 0.44966766610741615
Validation Loss: 0.4269372555116812
